In [6]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import sys
%matplotlib inline

from graph import *

# Review of graphs and adjacency matrices

How can we write the graph below as an adjacency matrix? 

In [7]:
make_graph_adj_random(4)

Remember that:

$A_{ij} = 1$ if the node $j$ has an outgoing edge (arrow) going to node $i$ 

$A_{ij} = 0$ otherwise

Hence the rows indicate incoming edges to a specific node and columns indicate outgoing edges from a specific node.

Write the adjacency matrix A of the above graph:

You can call the helper function:
```python    
graph.graph_matrix(mat, mat_label=None, show_weights=True, round_digits=3)
# mat: 2d numpy array of shape (n,n) with the adjacency matrix
# mat_label: 1d numpy array of shape (n,) with optional labels for the nodes
# show_weights: boolean - option to display the weights of the edges
# round_digits: integer - number of digits to display when showing the edge weights
```
to check if you get the same graph. Since in this example all the edge weights are zero, you should use `show_weights=False`

# Transition matrices

### Using graphs to represent the transition from one state to the other

After collecting data about the weather for many years, you observed that the chance of a rainy day occurring after one rainy day is 50% and that the chance of a rainy day after one sunny day is 10%. 

The graph that represents the transition from the weather on day 1 to the weather on day 2 can be expressed as an adjacency matrix, where the edge weights are the probabilities of weather conditions. We call that the transition matrix.

Write the transition matrix ${\bf A}$ for the weather observation described above, and use the helper function to plot the graph. Your graph will look better if you assign the labels for your nodes, for example, use the label `['sunny','rainy']`


### Properties of a transition (Markov) matrix

- $A_{ij}$ entry of a transition matrix has the probability of transitioning from state $j$ to state $i$

- Since the entries are probabilities, they are always non-negative real numbers, and the columns should add up to one.


### The weather today is sunny. What is the probability of sunny day tomorrow?

The answer to this question is quite trivial, and in this example, we can directly get that information from the graph.
But how could we obtain the same as a matrix-vector multiplication?

Write the numpy array ${\bf x0}$ representing the current state, which is the weather today. Your vector should be consistent with the transition matrix. If the first column of ${\bf A}$ corresponded to transitioning **from** a sunny day, the first entry of the state vector should be sunny.

In [12]:
x0 = np.array([1,0])

You can now obtain the probabilities for tomorrow weather by performing a matrix-vector multiplication:

In [13]:
x1 = A@x0
print(x1)

### The weather today (Thursday) is sunny. What is the probability of rain on Sunday? Should I keep my plans for a barbecue?

### What is the probability of sunny days in the long run? 

Let's run power iteration for the equivalent of 20 days. We will store all the state vectors ${\bf x}$ as columns of a matrix, so that we can plot the results later.

We will store all iterations in `allx` in order to view the running probabilities later.

In [19]:
its = 20
allx = np.zeros((2,its))

We will start with the initial state of rainy day:

In [20]:
x = np.array([0.5,0.5])
allx[:,0] = x

What are the probabilities after 20 days? Store the remaining columns of `allx`

In [22]:
print('Probabilities of initial state:', allx[:,0])
print('Probabilities after 20 days:', allx[:,-1])
plt.plot(allx.T)
plt.xlabel('')

What if we were to start with a random current state? Would the weather probabilities change in the long run? Write a similar code snippet, but now starting at a random initial state. Remember that the sum of the probabilities has to be equal to 1 (or that columns and state vectors should sum to 1). Think about normalizing. Which norm would satisfy this property?

In [23]:
x = np.random.rand(2)
x = x/la.norm(x,1)
print(x)

its = 20
allx = np.zeros((2,its))
allx[:,0] = x

for k in range(1,its):
    x = A@x
    allx[:,k] = x
    
print(x)

In [24]:
print('Probabilities of initial state:', allx[:,0])
print('Probabilities after 20 days:', allx[:,-1])
plt.plot(allx.T)
plt.xlabel('')

You can run your code snippet above a few times. What do you notice?

### Summarizing:

- You started with an initial weather state ${\bf x_0}$.

- Using the transition matrix, you can get the weather probability for the next day: ${\bf A  x_0} = {\bf x_1}$.

- In general, you can write ${\bf A  x_n} = {\bf x_{n+1}}$.

- Predictions for the weather on more distant days are increasingly inaccurate.

- Power iteration will converge to a **steady-state** vector, that gives the weather probabilities in the long-run.

$${\bf A  x^*} = {\bf x^*}$$

- ${\bf x^*}$ is the long-run equilibrium state, or the eigenvector corresponding to the eigenvalue $\lambda = 1$.

- This steady-state condition ${\bf x^*}$ does not depend on the initial state.

# Example 2: What are students doing during CS 357 lectures?

Consider the following graph of states. Assume this is a model of the behavior of a student after 10 minutes of a lecture. For example, if a student at the beginning of a lecture is participating in the class activity, there is a probability of 20% that they will be surfing the web after 10 minutes.

<img src="StudentLecture.png" style="width: 500px;"/>

Write the transition matrix ${\bf A}$ corresponding to the graph above. Build your matrix so that the columns are given in the following order:
```python
activity_names = ['lecture', 'web', 'hw', 'text']
```
Plot the graph using the `graph_matrix` helper function, to make sure you are indeed building the correct matrix. You can use the list above to label the nodes. 

In [25]:
activity_names = ['lecture', 'web', 'hw', 'text']

A  = np.array([
    [0.6,  0.4, 0.2, 0.3],
    [0.2, 0.5, 0.1, 0.2],
    [0.15, 0.1,  0.7, 0.0],
    [0.05, 0, 0  , 0.5]
])

graph_matrix(A,activity_names)

Build the initial state vector ${\bf x}$. Recall that this array should follow the same order as the matrix, here defined in `activity_names`.  In the initial state (suppose that we are at the 0 minute mark of the lecture, i.e. the beginning of the lecture), students have an 80% probability of participating in lecture, 10% probability of texting friends and 10% probability of surfing the web. 

In [26]:
x0 = np.array([0.8,0.1,0,0.1])

What is the probability that students will be working on their HW after 10 minutes?

In [27]:
x = x0
# Update x
print(x)
print("Probability of working on HW after 10 minutes is:", x[2])

What is the probability that students will be sending text messages after 30 minutes? Start from the random state provided below:

In [29]:
x = np.random.random(4)
x = x/la.norm(x,1)

# Update x
    
print("Probability of surfing the web after 30 minutes is:", x[3])

Would your answer above change if you were to start from a different initial state? Try for example x0 = [0, 0.2, 0.1, 0.7].

Steady-state will not depend on the initial state, but before achieving steady-state, the probabilities will be different!

Get the steady-state for this problem:
- Start from a random initial state. 
- Make sure the probability of the initial state sums to 1. 
- Assume 20 iterations of power method

In [31]:
its = 20
allx = np.zeros((4,its))

In [32]:
x0 = np.random.rand(4)
x0 = x0/la.norm(x0,1)

x = np.copy(x0)
allx[:,0] = x

for k in range(1,its):
    x = A.dot(x)
    allx[:,k] = x
    
print(x)

In [33]:
plt.plot(allx.T)
plt.legend(activity_names)
plt.grid()

Do you think you achieved steady-state at iteration 40?

Note that iteration 40 corresponds to the 400th minute of a lecture, which in this case does not exist (luckily? :-))

Does the class achieve steady-state at the end of lecture (about 70 minutes or 7 steps)?

Use `numpy.linalg.eig` to see how the steady-state (solution of power iteration) is the eigenvector of ${\bf A}$ corresponding the the eigenvalue 1. The eigenvectors are given in the columns of ${\bf U}$.

In [34]:
lambdas, U = la.eig(A)
print(lambdas)
print(U[:,0])

Not what you expected? Remember that if ${\bf u}$ is an eigenvector of ${\bf A}$, then $\alpha {\bf u}$ is also en eigenvector. Can you show that the eigenvector from the power iteration solution is the same as the one obtained using `numpy.linalg.eig`?

In [36]:
uvec = U[:,0]
uvec = uvec/la.norm(uvec,1)
print(uvec)